In [ ]:
import os
import pandas as pd
from autoemulate.core.compare import AutoEmulate
import torch
figsize = (9, 5)

In [ ]:
param_filename = 'parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas'

n_samples = [512, 1024]

for i_n_samples in n_samples:

    print(f'Training emulators on {i_n_samples} samples')

    simulation_out_path = f'../outputs/simulations/output_{i_n_samples}_samples_{param_filename}/'

    parameters_json_file = os.path.join(simulation_out_path, 'parameters.json')

    # Load the simulation input parameters
    input_params = pd.read_csv(os.path.join(simulation_out_path, f'input_samples_{i_n_samples}.csv'))

    # Load the summary statistics
    summary_stats = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

    # Get column names for parameters up to 'lv.k_pas'
    parameter_names = list(input_params.columns[:input_params.columns.get_loc('lv.k_pas') + 1])

    # Turn x into a pytorch tensor
    x = torch.tensor(input_params[parameter_names].values, dtype=torch.float32)

    for i_output in summary_stats.columns:
        
        print(f'Processing {i_output}')

        # Create a directory for the emulators
        emulators_path = os.path.join(simulation_out_path, 'emulators', i_output)
        os.makedirs(emulators_path, exist_ok=True)

        # Get the target variable
        Y = summary_stats[i_output].values
        y = torch.tensor(Y, dtype=torch.float32)

        print(x.shape, y.shape)

        # Initialize the AutoEmulate class
        ae = AutoEmulate(x = x,
                         y = y,
                         models=["GP"])

        print(ae.summarise())

        best = ae.best_result()

        print(f'Best model is: {best.model_name}')

        # Save the best model
        print('Saving the best model...')
        ae.save(model_obj=best,
                path=emulators_path,
                use_timestamp=False)
        
        # Add cleanup at the end of inner loop
        del ae, y, Y
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Add cleanup at the end of outer loop
    del x, input_params, summary_stats
    import gc
    gc.collect()